In [3]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [4]:
def load_dataset():
    """
    Load training and test sets
    """

    Xmat_train = pd.read_csv("./dataset_twitter/train.csv")
    Xmat_test = pd.read_csv("./dataset_twitter/test.csv")
    
    Y_train = pd.read_csv("./dataset_twitter/Y_train.csv")
    Y_test = pd.read_csv("./dataset_twitter/Y_test.csv")
    
    return Xmat_train, Y_train, Xmat_test, Y_test

In [5]:
Xmat_train, Y_train, Xmat_test, Y_test = load_dataset()

In [9]:
Y_train

,Unnamed: 0,Y_soft,Y_hard
0,0,[0.01429202 0.10583829 0.8798697 ],[0. 0. 1.]
1,1,[0.61227626 0.36075017 0.02697357],[1. 0. 0.]
2,2,[0.08997538 0.7731337 0.13689099],[0. 0. 1.]
3,3,[0.31554261 0.5471459 0.13731146],[0. 1. 0.]
4,4,[0.00645745 0.32842988 0.66511261],[0. 1. 0.]
...,...,...,...
19231,19231,[0.00316286 0.00544586 0.9913913 ],[0. 0. 1.]
19232,19232,[0.01371885 0.53129822 0.45498297],[0. 1. 0.]
19233,19233,[0.43192446 0.48019388 0.08788162],[0. 1. 0.]
19234,19234,[0.27887401 0.6551649 0.06596117],[1. 0. 0.]


In [ ]:
# Assuming we have train and test sets
Xmat_train = 
Y_train = 

Xmat_test = 
Y_test =

# Create a Bag of Words representation using the training data *only*
vectorizer = CountVectorizer()
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

# Train a Logistic Regression model
baseline_bow = LogisticRegression(max_iter=1000, multi_class='auto', solver='lbfgs')
baseline_bow.fit(X_train_bow, y_train)

# Make predictions
Y_pred_train = baseline_bow.predict(X_train_bow)
Y_pred_test = baseline_bow.predict(X_test_bow)

# Evaluate the model
train_accuracy = accuracy_score(Y_train, Y_pred_train)
test_accuracy = accuracy_score(Y_test, Y_pred_test)

print(f"Train accuracy: {train_accuracy}")
print(f"Test accuracy: {test_accuracy}")
